# Multi-Agent Orchestration

In this notebook, you'll create a team of agents that can create detailed research reports.

Here's how this will work:

1. User ask question to lead agent
2. Lead agent decomposes question and creates research plan with multiple tasks
3. Lead agent calls research agent with task 1 instructions
4. Research agent examines task 1 instructions
5. Research agent calls `search_pmc` and `gather_evidence` tools multiple times. Every time `gather_evidence` finishes, it saves the evidence gathered from a single PMC article to DynamoDB
6. Research agent returns a task report with a summary and list of evidence id values
7. Lead agent examines task report and updates research plan with information from research agent
8. Repeat steps 3-7 for all tasks.
9. Once research is complete, lead agent submits each section 1 summary and evidence to technical writing agent
10. Technical writing agent retrieves evidence records from DynamoDB and submits section summary and evidence to Claude citation API
11. Claude generates report section with inline citations
12. Technical writing agent writes report section to file
13. Technical writing agent reports completion to lead agent
14. Repeat steps 9-13 for all sections
15. Lead agent uses publish tool to reformat citations and return document to user.

## 1. Prerequisites

- Python 3.12 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude 3.7 Sonnet model in Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U -r requirements.txt

In [2]:
import logging

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Enables Strands warnings log level
logging.getLogger("strands").setLevel(logging.WARNING)

In [3]:
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

## 2. Define PMC Research Agent

This agent will be similar to the gather evidence agent we created in notebook 2, but with a change in how the evidence is stored. To avoid the "game of telephone" problem, we are going to deterministically store the gathered evidence in a DynamoDB table. This is a big advantage to building AI agents on AWS - they can leverage any of the 200+ services to store and process data.

Let's test out the updated tool to see how it works.

First, we create a DynamoDB table, with `toolUseId` as the primary key and `pmcid` as a global secondary id. This gives us the flexibility to extract evidence records either by tool use or paper. 

In [4]:
import boto3

dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "deep-research-evidence"
try:
    table = dynamodb.Table(table_name)
    table.load()
    print(f"Table '{table_name}' already exists")
except:
    # Create table if it doesn't exist
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[{"AttributeName": "toolUseId", "KeyType": "HASH"}],
        AttributeDefinitions=[
            {"AttributeName": "toolUseId", "AttributeType": "S"},
            {"AttributeName": "pmcid", "AttributeType": "S"},
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "pmcid-index",
                "KeySchema": [{"AttributeName": "pmcid", "KeyType": "HASH"}],
                "Projection": {"ProjectionType": "ALL"},
            }
        ],
        BillingMode="PAY_PER_REQUEST",
    )

    table.wait_until_exists()

INFO | botocore.credentials | Found credentials in shared credentials file: ~/.aws/credentials


Table 'deep-research-evidence' already exists


Next, we run our agent with the updated `gather_evidence` tool and pass the db table name as an environment variable.

In [ ]:
from strands import Agent
from search_pmc import search_pmc_tool
from gather_evidence import gather_evidence_tool
import os

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
QUERY = "How safe and effective are GLP-1 drugs for long term use?"

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use search_pmc_tool with max_search_result_count between 200 and 500 and max_filtered_result_count between 10 and 20 to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 5 years"[dp] for recent work. 
2. Identify the PMC ID value for the most relevant paper, then submit the ID and the query to the gather_evidence_tool.
3. Generate a concise answer to the question based on the most relevant evidence, along with PMC ID and URL citations
"""

os.environ['EVIDENCE_TABLE_NAME'] = table_name


# Initialize your agent
pmc_research_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[search_pmc_tool, gather_evidence_tool],
    model=MODEL_ID,
)

# Send a message to the agent
response = pmc_research_agent(QUERY)

Let's look at the new records in our db table

In [5]:
dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "deep-research-evidence"
table = dynamodb.Table(table_name)

records = [record for record in table.scan().get('Items')]
records

[{'evidence': ['**Clinical recommendations for managing adverse events and ensuring safety during long‑term GLP‑1\u202fRA therapy**\n\n1. **Start low and titrate slowly** – Begin with the lowest available dose and increase gradually to the maintenance dose, allowing the gastrointestinal (GI) tract to adapt.\n\n2. **Monitor for GI adverse events (AEs)** – During dose escalation, watch for nausea, vomiting, diarrhoea, or constipation. If an AE appears:\n   * **Mild‑moderate**: keep the current dose, provide supportive measures (dietary adjustments, adequate hydration, small frequent meals, low‑fat foods) and consider anti‑emetic or antidiarrheal agents.\n   * **Severe or persistent**: pause dose escalation, reduce the dose, or temporarily discontinue until symptoms improve, then resume titration at a slower pace.\n\n3. **Preventive strategies**  \n   * **Nausea/vomiting** – Take medication with food, avoid large meals, limit fatty or spicy foods, and consider prophylactic anti‑emetics if

We can also query for a records from a specific PMC ID

In [ ]:
example_pmc_id = records[0]["pmcid"]
print(example_pmc_id)

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key
dynamodb = boto3.resource("dynamodb")
table_name = "deep-research-evidence"
table = dynamodb.Table(table_name)


response = table.query(
    IndexName="pmcid-index", KeyConditionExpression=Key("pmcid").eq(example_pmc_id)
)
example_db_records = response.get('Items')
example_db_records

We'll use our `pmc_research_agent` again later in this notebook

## 3. Define Technical Writing Agent

Next, we'll build our writing agent. This agent will take in a question and citations and use the Anthropic Claude citations API to add citations. Let's test how this works with the Amazon Bedrock InvokeModel API 

Ref: https://docs.claude.com/en/docs/build-with-claude/citations

### 3.1. Explore Anthropic Claude Citations API

Here is an example of how to use the Claude citations API through a Bedrock InvokeModel call

In [ ]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Based on the available context, specific recommendations for managing gastrointestinal adverse events during GLP-1 receptor agonist (GLP-1 RA) therapy include several key strategies. Management emphasizes comprehensive patient education, initiating treatment with low starting doses, and implementing gradual dose titration to minimize adverse events (Gorgojo2023 chunk 2). Dietary adjustments and ongoing monitoring are also essential components of the management approach (Gorgojo2023 chunk 2).\n\nRegarding clinical experience with long-term safety, GI adverse events occur in 40-85% of patients receiving GLP-1 RAs, but these events are typically mild and transient in nature (Gorgojo2023 chunk 2). The adverse events generally resolve following dose escalation, indicating that patients can adapt to therapy over time (Gorgojo2023 chunk 2). \n\nLong-term safety data demonstrate a favorable profile, with most events classified as non-serious and low rates of permanent treatment discontinuation (Gorgojo2023 chunk 2). This clinical experience supports the overall safety profile of GLP-1 RAs for long-term use (Gorgojo2023 chunk 2). The combination of appropriate management strategies and the generally mild, self-limiting nature of GI adverse events allows most patients to continue therapy successfully over extended periods.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Long-term GLP-1 receptor agonist therapy demonstrates sustained efficacy in maintaining glycemic control and promoting weight loss (Zheng2024 chunk 55). However, therapeutic effectiveness may plateau over extended treatment periods, with studies indicating this limitation is associated with increased orbitofrontal reward activation, as observed with liraglutide (Zheng2024 chunk 55).\n\nThe adverse event profile is predominantly gastrointestinal, with nausea and vomiting representing the most common side effects (Zheng2024 chunk 55). Serious but rare adverse events include pancreatitis and gallbladder disease (Zheng2024 chunk 55).\n\nSeveral absolute contraindications exist for long-term GLP-1 receptor agonist use. Patients with a personal or family history of medullary thyroid carcinoma should not receive these agents (Zheng2024 chunk 55). Multiple endocrine neoplasia type 2 (MEN 2) also represents a contraindication (Zheng2024 chunk 55). Additionally, severe gastrointestinal disorders preclude the use of these medications (Zheng2024 chunk 55).\n\nThe safety and efficacy profile supports long-term use in appropriate patients, though careful patient selection is essential given the specific contraindications, particularly those related to thyroid malignancy risk and pre-existing gastrointestinal pathology.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },                
                {
                    "type": "text",
                    "text": "What is the long-term safety profile and effectiveness of GLP-1 receptor agonists? What are the main adverse events and contraindications for long-term use?",
                },
            ],
        }
    ],
    "max_tokens": 1024,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print(response_body.get('content'))

We can now format the response into a citated output

In [ ]:
def print_cited_response(response_content: dict) -> None:
    citations = []
    for content_item in response_content:
        print(content_item.get("text"), end="")
        for citation in content_item.get("citations", []):
            title = citation.get("document_title")
            if title not in citations:
                citations.append(title)
            print(f" ({citations.index(title)+1})", end="")

    print("\n")
    print("## References")
    for i, title in enumerate(citations, start=1):
        print(f"{i}. https://www.ncbi.nlm.nih.gov/pmc/articles/{title}")
    return None

print_cited_response(response_body.get('content'))

Let's try it with an example record from our db. In this case, we're using a custom content type to pass all of the gathered evidence as a single, citable block. We also include the generated answer as additional context (this won't be cited)

In [ ]:
# Defined above
example_db_records 

In [ ]:
import boto3
import json

def parse_db_records(records):
    """ Parse records from our DynamoDB table into content blocks for the Anthropic Claude citation API"""

    contents = []
    for record in records:
        contents.append(
            {
                "type": "document",
                "source": {
                    "type": "content",
                    "content": [
                        # In this case, we join all of the evidence items together.
                        {"type": "text", "text": "".join(record.get("evidence"))}
                        # If we wanted to reference individual chunks, we could also do something like:
                        # {"type": "text", "text": evidence} for evidence in record.get("evidence")
                    ],
                },
                "title": record.get("pmcid"),
                "context": record.get("answer"),
                "citations": {"enabled": True},
            },
        )

    contents.append(
        {
            "type": "text",
            "text": "What is the long-term safety profile and effectiveness of GLP-1 receptor agonists? What are the main adverse events and contraindications for long-term use?",
        },
    )
    return(contents)


In [ ]:
contents = parse_db_records(example_db_records)

print("Bedrock InvokeModel request contents are:\n")
for i in contents:
    print(i)
print()

bedrock_client = boto3.client("bedrock-runtime")

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": contents}],
    "max_tokens": 1024,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print_cited_response(response_body.get('content'))

Let's try one more time, this time with a system prompt with the writing guidelines we gave to our agent in notebook 3

In [ ]:
from datetime import date

SYSTEM_PROMPT = f"""
The current date is {date.today().strftime('%B %d, %Y')}

You are an expert technical writer that answers biomedical questions using scientific literature and other authoritative sources. 
You maintain user trust by being consistent (dependable or reliable), benevolent (demonstrating good intent, connectedness, and care), transparent (truthful, humble, believable, and open), and competent (capable of answering questions with knowledge and authority).
Use a professional tone that prioritizes clarity, without being overly formal.
Use precise language to describe technical concepts. For example, use, "femur" instead of "leg bone" and "cytotoxic T lymphocyte" instead of "killer T cell".

Structure your output as a comprehensive document that clearly communicates your research findings to the reader. Follow these guidelines:

Report Structure:

- Begin with a concise introduction (1-2 paragraphs) that establishes the research question, explains why it's important, and provides a brief overview of your approach
- Organize the main body into sections that correspond to the major research tasks you completed (e.g., "Literature Review," "Current State Analysis," "Comparative Assessment," "Technical Evaluation," etc.)
- Conclude with a summary section (1-2 paragraphs) that synthesizes key findings and discusses implications

Section Format:

- Write each section in paragraph format using 1-3 well-developed paragraphs
- Each paragraph should focus on a coherent theme or finding
- Use clear topic sentences and logical flow between paragraphs
- Integrate information from multiple sources within paragraphs rather than listing findings separately

Citation Requirements:

- Include proper citations for all factual claims using the format provided in your source materials
- Place citations at the end of sentences before punctuation (e.g., "Recent studies show significant progress in this area .")
- Group related information from the same source under single citations when possible
- Ensure every major claim is supported by appropriate source attribution

Writing Style:

- Use clear, professional academic language appropriate for scientific communication
- Use active voice and strong verbs
- Synthesize information rather than simply summarizing individual sources
- Draw connections between different pieces of information and highlight patterns or contradictions
- Focus on analysis and interpretation, not just information presentation
- Don't use unnecessary words. Keep sentences short and concise.
- WRite for a global audience. Avoid jargon an colloquial language. 

Quality Standards:

- Ensure logical flow between sections and paragraphs
- Maintain consistency in terminology and concepts throughout
- Provide sufficient detail to support conclusions while remaining concise
- End with actionable insights or clear implications based on your research findings
"""


In [ ]:
bedrock_client = boto3.client("bedrock-runtime")

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": parse_db_records(example_db_records)}],
    "max_tokens": 2048,
    "system": SYSTEM_PROMPT,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print_cited_response(response_body.get("content"))

### 3.2. Create citation tool